Attempting to load the data descriptor for the occurrence text file.

In [1]:
import os
import pandas as pd
import sys
from collections import Counter

from dwca.read import DwCAReader
from dwca.descriptors import shorten_term
from dwca.darwincore.utils import qualname as qn

In [2]:
home_dir = os.path.dirname(os.getcwd())
# data_dir = "/Users/lbokeria/Documents/projects/gbif-species-trainer-data/"
data_dir = "/bask/projects/v/vjgo8416-amber/data/gbif-species-trainer-AMI-fork/"

family_name = "Sesiidae"

dwca_file_path       = os.path.join(data_dir,"dwca_files",family_name+".zip")
occurrence_file_path = os.path.join(data_dir,"dwca_files",family_name,"occurrence.txt")

# Remove the extra rows and compare the size
fields_to_keep = [
    "id",
    "decimalLatitude",
    "decimalLongitude",
    "order",
    "family",
    "genus",
    "species",
    "acceptedScientificName",
    "year",
    "month",
    "day",
    "datasetName",
    "taxonID",
    "acceptedTaxonKey",
    "lifeStage",
    "basisOfRecord",
]

In [3]:
kwargs = {}
with DwCAReader(dwca_file_path) as dwca:
    # Get the file descriptor
    datafile_descriptor = dwca.get_descriptor_for("occurrence.txt")
    
    kwargs['delimiter'] = datafile_descriptor.fields_terminated_by
    kwargs['skiprows'] = datafile_descriptor.lines_to_ignore
    kwargs['header'] = None
    kwargs['names'] = datafile_descriptor.short_headers
    kwargs['parse_dates'] = True
    kwargs['on_bad_lines'] = "skip"
    kwargs['usecols'] = fields_to_keep    
    
    df1 = pd.read_csv(occurrence_file_path, **kwargs)
    
    # Add a column for default values, if present in the file descriptor
    for field in datafile_descriptor.fields:
        field_default_value = field['default']
        if field_default_value is not None:
            df1[shorten_term(field['term'])] = field_default_value

/tmp/ipykernel_4138161/2301844714.py:14: DtypeWarning: Columns (61,76,180) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(occurrence_file_path, **kwargs)


In [4]:
# Read normally and compare
with DwCAReader(dwca_file_path) as dwca:
    df2 = dwca.pd_read("occurrence.txt", parse_dates=True, on_bad_lines="skip",usecols=fields_to_keep)

/bask/projects/v/vjgo8416-amber/conda_envs/gbif-species-trainer-AMI-fork/lib/python3.8/site-packages/dwca/read.py:203: DtypeWarning: Columns (61,76,180) have mixed types. Specify dtype option on import or set low_memory=False.
  df = read_csv(self.absolute_temporary_path(relative_path), **kwargs)


In [5]:
# Compare the dfs

df1.compare(df2)

Empty DataFrame
Columns: []
Index: []